# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
root = document.getroot()

In [7]:
root.tag

'mondial'

In [8]:
root.attrib

{}

In [9]:
root.text

'\n   '

In [43]:
# print names of all countries and their cities
country = []
infantm = []
for element in document.iterfind('country'):
    country.append(element.find('name').text + ':')
    #capitals_string = ''
    try:
        infantm.append(element.find('infant_mortality').text)
    except:
        infantm.append(np.nan)
    #for subelement in element.getiterator('city'):
        #capitals_string += subelement.find('').text + ', '
    #print capitals_string[:-2]

In [44]:
document.iterfind('country')

<generator object select at 0x000000000B27CE10>

In [45]:
import pandas as pd
import numpy as np


In [46]:
df = pd.DataFrame({
         "country" : country, "infant_mortality": infantm
    })

In [47]:
df.dtypes

country             object
infant_mortality    object
dtype: object

In [48]:
df['infant_mortality'] = df.infant_mortality.astype(np.float)

In [49]:
#Answer 1
df_clean = df[df.infant_mortality.notnull()]
df_clean.sort_values(by = 'infant_mortality').head(10)

,country,infant_mortality
38,Monaco:,1.81
98,Japan:,2.13
117,Bermuda:,2.48
36,Norway:,2.48
106,Singapore:,2.53
37,Sweden:,2.60
10,Czech Republic:,2.63
78,Hong Kong:,2.73
79,Macao:,3.13
44,Iceland:,3.15


In [22]:
city = []
population = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        city.append(subelement.find('name').text + ':')
        try:
            population.append(subelement.find('population').text)
        except:
            population.append(np.nan)

In [23]:
pop = pd.DataFrame({
         "city" : city, "population": population
    })

In [24]:
pop.dtypes

city          object
population    object
dtype: object

In [25]:
pop['population'] = pop.population.astype(np.float)

In [26]:
pop.head()

,city,population
0,Tirana:,192000.0
1,Shkodër:,62000.0
2,Durrës:,60000.0
3,Vlorë:,56000.0
4,Elbasan:,53000.0


In [27]:
#Answer 2
pop_clean = pop[pop.population.notnull()]
pop_clean.sort_values(by = 'population').tail(10)

,city,population
1582,Delhi:,7206704.0
1340,Beijing:,7362426.0
1876,Tokyo:,7843000.0
479,Moskva:,8010954.0
2109,Ciudad de México:,8092449.0
1341,Shanghai:,8205598.0
1757,Jakarta:,8259266.0
2810,São Paulo:,9412894.0
1527,Mumbai:,9925891.0
1928,Seoul:,10229262.0


In [39]:
#Answer to Question 3
ethnic_group_population_by_country = []

for element in document.getiterator('country'):
    # Get (population year, population value) tuples for the country
    population_by_year = ((int(population.attrib['year']),
                          int(population.text))
                          for population in element.iter('population'))
    
    # Get the last population value
    # This work because max sort tuples by first element first
    population_most_recent = max(population_by_year)[1]
    
    # Get (ethnic group, population) tuples
    # We're calculating ethnic great population as
    # percentage of total population
    ethnic_group_population = [
        (ethnic_group.text,
         population_most_recent * 0.01 * float(ethnic_group.attrib['percentage']))
        for ethnic_group in element.iter('ethnicgroup')
    ]
    
    # Append this list to our larger list
    ethnic_group_population_by_country += ethnic_group_population

In [40]:
ethnic_group_by_country_df = pd.DataFrame.from_records(
    ethnic_group_population_by_country,
    columns=['ethnicgroup', 'population'])
ethnic_group_by_country_df.head()

,ethnicgroup,population
0,Albanian,2660131.100
1,Greek,84004.140
2,Greek,216.690
3,Macedonian,1322387.748
4,Albanian,519068.088


In [41]:

# Group by ethnic group and aggregate
ethnic_group_by_country_df.groupby('ethnicgroup').agg(sum) \
.sort_values(by='population', ascending=False)[:10]

,population
ethnicgroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.305192e+08
Japanese,1.265342e+08


In [121]:
river = []
length = []
country = []
for element in document.iterfind('river'):
    for located in element.iter('river'):
        country.append(located.attrib['country'])
    for subelement in element.getiterator('river'):                     
        river.append(subelement.find('name').text)
        try:
            length.append(subelement.find('length').text)
        except:
            length.append(np.nan)

In [122]:
rivers = pd.DataFrame({
         "country" : country, "river": river, "length": length
    })

In [123]:
rivers['length'] = rivers.length.astype(np.float)

In [126]:
#Longest River is Amazonas in Colombia, Brazil, and Peru
rivers.sort_values(by = 'length', ascending= False)[:1]


,country,length,river
174,CO BR PE,6448.0,Amazonas


In [151]:
country_code[country_code.code == 'CO']

,code,country
141,CO,Colombia


In [152]:
country_code[country_code.code == 'BR']

,code,country
176,BR,Brazil


In [153]:
country_code[country_code.code == 'PE']

,code,country
180,PE,Peru


In [136]:
lake = []
size = []
country = []
for element in document.iterfind('lake'):
    for located in element.iter('lake'):
        country.append(located.attrib['country'])
    for subelement in element.getiterator('lake'):                  
        lake.append(subelement.find('name').text)
        try:
            size.append(subelement.find('area').text)
        except:
            size.append(np.nan)

In [137]:
lakes = pd.DataFrame({
         "country" : country, "lake": lake, "size": size
    })

In [138]:
lakes['size'] = lakes.size.astype(np.float)

In [140]:
#Largest lake is Inari in Finland
lakes.sort_values(by = 'size', ascending= False)[:1]

,country,lake,size
0,SF,Inari,423.0


In [144]:
code = []
country_name = []
for element in document.iterfind('country'):
    for country in element.iter('country'):
        code.append(country.attrib['car_code'])                 
        country_name.append(element.find('name').text)

In [146]:
country_code = pd.DataFrame({
         "country" : country_name, "code": code
    })

In [150]:
country_code[country_code.code == 'SF']

,code,country
35,SF,Finland


In [155]:
airport = []
elevation = []
country = []
for element in document.iterfind('airport'):
        country.append(located.attrib['country'])                
        airport.append(element.find('name').text)
        elevation.append(element.find('elevation').text)
       

In [156]:
airports = pd.DataFrame({
         "country" : country, "airport": airport, "elevation": elevation
    })

In [157]:
airports['elevation'] = airports.elevation.astype(np.float)

In [158]:
#Highest Airport is El Alto Intl in Australia
airports.sort_values(by = 'elevation', ascending= False)[:1]

,airport,country,elevation
80,El Alto Intl,AUS,4063.0


In [159]:
country_code[country_code.code == 'AUS']

,code,country
151,AUS,Australia
